Test of station beamformer
Sets up a station with 2 tiles (T1 nd T4, could be different) and programs it to send 
beamformed data to a server, for 8 contiguous channels.

Server runs a DAQ receiver in station beam data mode, which computes total power on blocks of 262144 samples. 

Station beamformer is programmed and configured to generate either a tone or white noise on the selected channels

Delay is inserted on each antenna to simulate a linear arrays of 32 stations which is then pointed using first a mosaic scan (individual pointings) and then a on-the-fly scan (initial pointing and delay rates).

A test on decorrelation is performed by adding a positive and negative static delay on the two tiles, and correctling it using pointing delays.

Initial setup. Define the used TANGO devices and constants. 

Put all devices Online. Tiles must be in Maintenance mode in order to use the test signal generator.

In [1]:
import tango
import time
import json
import numpy as np

from ska_tango_base.commands import ResultCode
from ska_tango_base.control_model import (
    AdminMode,
    CommunicationStatus,
    HealthState,
    PowerState,
    SimulationMode,
    TestMode,
)
# for time conversion
from datetime import datetime,timezone
RFC_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"

# define devices
station = tango.DeviceProxy('low-mccs/station/001')
subrack = tango.DeviceProxy('low-mccs/subrack/0001')
t1 = tango.DeviceProxy('low-mccs/tile/0001')
t2 = tango.DeviceProxy('low-mccs/tile/0002')
t3 = tango.DeviceProxy('low-mccs/tile/0003')
t4 = tango.DeviceProxy('low-mccs/tile/0004')
t5 = tango.DeviceProxy('low-mccs/tile/0005')
t6 = tango.DeviceProxy('low-mccs/tile/0006')
t7 = tango.DeviceProxy('low-mccs/tile/0007')
t8 = tango.DeviceProxy('low-mccs/tile/0008')

station.logginglevel=5
devices = [station, subrack, t1, t2, t3, t4, t5, t6, t7, t8]
tiles = [t1, t4]
#
# Put everything online
for d in devices:
    d.adminmode = AdminMode.ONLINE
time.sleep(0.2)
# 
# Tiles must be in Maintenance mode to allow test signal generator
for t in tiles:
    t.adminMode = AdminMode.MAINTENANCE

Put the station in standby mode. Wait 5 seconds for the command to take effect.

In [2]:
station.standby()
time.sleep(5)

Set all the Station parameters, like destination IP addresses, channeliser and final beamformer rounding. 

Initialise everything by torning the station on. This powers on the tiles and initialises them. 

Wait until all tiles are Initialised

In [3]:
station.SetBeamformerTable([128,0,1,0,0,0,0])
station.statictimedelays=np.zeros([512],dtype=int)
station.preaduLevels=list(range(32))*16
station.channeliserRounding=[4]*512
station.cspRounding=[4]*384
station.SetLmcDownload('{"destination_ip": "10.0.0.98", "mode": "40g"}')
station.SetLmcIntegratedDownload('{"destination_ip": "10.0.0.98", "mode": "40g"}')
station.SetCspIngest('{"destination_ip": "10.0.0.98"}')
# turn everything on
print(f"Station: {station.Status()}")
station.on()
#
# monitor what is happening
#
state = station.tileprogrammingstate
tm = 0
init = False
for t in range (30):
    time.sleep(2)
    tm = tm + 2
    print(f't={tm}: subrack: {subrack.status()}')
    if subrack.state() == tango._tango.DevState.ON:
        break
for t in range(30):
    time.sleep(2)
    tm = tm + 2
    s_new = station.tileprogrammingstate
    if s_new != state:
        print(f't={tm}: state = {s_new}')
        state = s_new
    if all(s == 'Initialised' for s in state):
        init = True
        break
if init:
    print(f't={tm}: Station initialized')
else:
    print(f't={tm}: Timeout during intialisation')

Station: The device is in STANDBY state.
t=2: subrack: The device is in ON state.
t=10: state = ('NotProgrammed', 'Off')
t=14: state = ('NotProgrammed', 'NotProgrammed')
t=18: state = ('Programmed', 'NotProgrammed')
t=22: state = ('Programmed', 'Programmed')
t=38: state = ('Initialised', 'Programmed')
t=40: state = ('Initialised', 'Initialised')
t=40: Station initialized


Synchronise the station. 
Then wait for the synchronization to happen

In [4]:
start_time = datetime.strftime(datetime.fromtimestamp(int(time.time())+3), RFC_FORMAT)
station.StartAcquisition(json.dumps({
  "start_time": start_time}))
#
# check that synchronization worked
#
print(f'Tile time: {t1.fpgatime} - Sync time: {start_time}')
print(f'Programmed Sync time: {t1.fpgareferencetime}')
time.sleep(1)
for t in range (1):
    tm1 = datetime.strftime(datetime.fromtimestamp(time.time()), RFC_FORMAT)
    tm2 = t1.fpgatime
    tm3 = t1.fpgaframetime
    print(f'time:{tm1} pps time:{tm2} frame time:{tm3}')
    #time.sleep(2)
for i in range(30):
    tm = tm + 1
    cur_time=int(t1.readregister('fpga1.pps_manager.curr_time_read_val')[0])
    start_time=int(t1.readregister('fpga1.pps_manager.sync_time_val')[0])
    print(f'Current: {cur_time} - Start: {start_time} difference: {cur_time-start_time} frame time:{t1.fpgaframetime}')
    if cur_time > start_time:
        break
    time.sleep(1)

tm1 = datetime.strftime(datetime.fromtimestamp(time.time()), RFC_FORMAT)
tm2 = t1.fpgatime
tm3 = t1.fpgaframetime
print(f'time:{tm1} pps time:{tm2} frame time:{tm3}')
print(f't={tm}: state = {station.tileprogrammingstate}')

Tile time: 2023-03-24T14:43:58.000000Z - Sync time: 2023-03-24T14:44:01.000000Z
Programmed Sync time: 2106-02-07T06:28:15.000000Z
time:2023-03-24T14:43:59.359374Z pps time:2023-03-24T14:43:59.000000Z frame time:2023-03-24T14:44:01.000000Z
Current: 1679669039 - Start: 1679669041 difference: -2 frame time:2023-03-24T14:44:01.000000Z
Current: 1679669040 - Start: 1679669041 difference: -1 frame time:2023-03-24T14:44:01.000000Z
Current: 1679669041 - Start: 1679669041 difference: 0 frame time:2023-03-24T14:44:01.507064Z
Current: 1679669042 - Start: 1679669041 difference: 1 frame time:2023-03-24T14:44:02.524234Z
time:2023-03-24T14:44:02.416460Z pps time:2023-03-24T14:44:02.000000Z frame time:2023-03-24T14:44:02.526999Z
t=44: state = ('Synchronised', 'Synchronised')


Program the test generator and start the beamformer

In [5]:
noise = True

for t in tiles:
    tm = t.fpgaframetime
    print(f"{t.name()}: {tm}")
start_time = datetime.strftime(datetime.fromtimestamp(time.time()+2), RFC_FORMAT)

if noise:
    json_arg=json.dumps({'noise_amplitude': 1.0, 'set_time': start_time})
else:
    json_arg=json.dumps({'tone_2_frequency': 100.01e6, 'tone_2_amplitude': 0.5, 'set_time': start_time})

for t in tiles:
    t.ConfigureTestGenerator(json_arg)
station.StartBeamformer(json.dumps({"start_time": start_time}))
time.sleep(2)
print(f"Beamformer running: {t1.isBeamformerRunning}")

low-mccs/tile/0001: 2023-03-24T14:44:10.207061Z
low-mccs/tile/0004: 2023-03-24T14:44:10.208719Z
Beamformer running: True


Send a few packets to the DAQ

In [6]:
# Send beamformed
start_time = datetime.strftime(datetime.fromtimestamp(time.time()+0.5), RFC_FORMAT)
station.SendDataSamples(json.dumps({'data_type': 'beam', 'start_time': start_time}))
time.sleep(2)
start_time = datetime.strftime(datetime.fromtimestamp(time.time()+0.5), RFC_FORMAT)
station.SendDataSamples(json.dumps({'data_type': 'channel', 'start_time': start_time}))

[array([0], dtype=int32), ['SendDataSamples command completed OK']]

<b>Scan test.</b>

Program static delays to a ramp of integer samples. This corresponds to physical delays in the range +/- 20 ns. Put static delays only in polarization 0, polarization 1 used as reference. 

Then point in steps of 1/40 ns per antenna (+/- 0.4 ns at array edge), from -8 to +32 ns 

In [7]:
static_delays = np.zeros([512],dtype=float)
d0 = 1.25
for i in range(32):
    delay =  (i-16)*d0
    static_delays[2*i+0] = delay
    static_delays[2*i+1] = 0
station.statictimedelays = static_delays
delays = np.zeros([513],dtype=float)
station.LoadPointingDelays(delays)
start_time = datetime.strftime(datetime.fromtimestamp(time.time()+0.5), RFC_FORMAT)
station.applypointingdelays(start_time)
time.sleep(3)
for step in range(-20,80):
    delays = np.zeros([513],dtype=float)
    d1 = 1.25*step/40
    for i in range(32):
        delay =  (i-16)*d1
        delays[2*i+1] = delay*1e-9
        delays[2*i+2] = 0.0
    delays[0] = 0
    station.LoadPointingDelays(delays)
    start_time = datetime.strftime(datetime.fromtimestamp(time.time()+0.5), RFC_FORMAT)
    station.applypointingdelays(start_time)
    time.sleep(1)
    if step%5 == 0:
        print(f"Delay offset {d1-d0} {d1}")

delays = np.zeros([513],dtype=float)   
station.LoadPointingDelays(delays)
start_time = datetime.strftime(datetime.fromtimestamp(int(time.time())+4), RFC_FORMAT)
station.applypointingdelays(start_time)

Delay offset -1.875 -0.625
Delay offset -1.71875 -0.46875
Delay offset -1.5625 -0.3125
Delay offset -1.40625 -0.15625
Delay offset -1.25 0.0
Delay offset -1.09375 0.15625
Delay offset -0.9375 0.3125
Delay offset -0.78125 0.46875
Delay offset -0.625 0.625
Delay offset -0.46875 0.78125
Delay offset -0.3125 0.9375
Delay offset -0.15625 1.09375
Delay offset 0.0 1.25
Delay offset 0.15625 1.40625
Delay offset 0.3125 1.5625
Delay offset 0.46875 1.71875
Delay offset 0.625 1.875
Delay offset 0.78125 2.03125
Delay offset 0.9375 2.1875
Delay offset 1.09375 2.34375


[array([0], dtype=int32), ['LoadPointingDelays command completed OK']]

Decorrelation test. Increase the delay in each tile up to 50 additional ns.

Then correct with pointing delays

In [8]:
for offset in range(40):
    static_delays=np.zeros([512],dtype=float)
    station.statictimedelays = static_delays
    for i in range(16):
       static_delays[2*i] = -1.25*offset
       static_delays[2*i+32] = 1.25*offset
    for i in range(32):
       static_delays[2*i] += (i-16)*1.25
    static_delays[0] = 1.25*offset
    static_delays[18] = 1.25*offset
    station.statictimedelays = static_delays
    delays = np.zeros([513],dtype=float)
    for i in range(32):
        delay = static_delays[2*i]*1.00e-9
        delays[2*i+1] = delay
        delays[2*i+2] = 0.0
    delays[0] = 0
    station.LoadPointingDelays(delays)
    start_time = datetime.strftime(datetime.fromtimestamp(time.time()+0.4), RFC_FORMAT)
    station.applypointingdelays(start_time)
    time.sleep(2)

<b> Drift scan test</b>

With the same delays as in the scan test, program delays and delay rates to scan across the same delay interval. <b>Test lasts approx. 1 hour. </b> Test is performed in hardware by the tile, programming returns immediately

In [9]:
static_delays = np.zeros([512],dtype=float)
d0 = 1.25
for i in range(32):
    delay =  (i-16)*d0
    static_delays[2*i+0] = delay
    static_delays[2*i+1] = 0
station.statictimedelays = static_delays

delays = np.zeros([513],dtype=float)
d1 = -1.25e-9
for i in range(32):
    delay =  (i-16)*d1
    delays[2*i+1] = delay
    delays[2*i+2] = -delay/1200.0
delays[0] = 0
station.LoadPointingDelays(delays)
start_time = datetime.strftime(datetime.fromtimestamp(time.time()+0.5), RFC_FORMAT)
station.applypointingdelays(start_time)

[array([0], dtype=int32), ['LoadPointingDelays command completed OK']]

Here one should wait 1 hour for the scan to complete.

Then test wrap down. Points to zenith, stops beamformer and switches off TPMs

In [10]:
static_delays=np.zeros([512],dtype=float)
station.statictimedelays = static_delays 
delays = np.zeros([513],dtype=float)
station.LoadPointingDelays(delays)
start_time = datetime.strftime(datetime.fromtimestamp(int(time.time())+0.5), RFC_FORMAT)
station.applypointingdelays(start_time)

[array([0], dtype=int32), ['LoadPointingDelays command completed OK']]

In [11]:
station.stopbeamformer()

[array([0], dtype=int32), ['StopBeamformer command completed OK']]

In [28]:
station.standby()

[array([2], dtype=int32), ['1679559608.8023124_251448388425952_Standby']]

In [30]:
t1.status()

'The device is in OFF state.'

In [31]:
station.status()

'The device is in STANDBY state.'